In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,5127
2,Huelva,Confirmados PDIA 14 días,2359
3,Huelva,Tasa PDIA 14 días,"461,8761294819508"
4,Huelva,Confirmados PDIA 7 días,1133
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,38
629,Municipio de Huelva sin especificar,Total Confirmados,100
630,Municipio de Huelva sin especificar,Curados,20


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  5127.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1357.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 501 personas en los últimos 7 días 

Un positivo PCR cada 226 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,5127.0,1133.0,2359.0,510743.0,221.833682,461.876129,152.0
Huelva-Costa,2568.0,546.0,1154.0,288115.0,189.507662,400.534509,84.0
Huelva (capital),1357.0,287.0,636.0,143663.0,199.773080,442.702714,43.0
Condado-Campiña,1682.0,386.0,789.0,155057.0,248.940712,508.845134,39.0
Sierra de Huelva-Andévalo Central,777.0,163.0,349.0,67571.0,241.227746,516.493762,28.0
Aljaraque,194.0,39.0,92.0,21260.0,183.443086,432.737535,11.0
Valverde del Camino,156.0,48.0,103.0,12820.0,374.414977,803.432137,10.0
Lepe,386.0,90.0,173.0,27431.0,328.095950,630.673326,8.0
Moguer,101.0,24.0,38.0,22088.0,108.656284,172.039116,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,78.0,32.0,56.0,2284.0,1401.050788,2451.838879,2.0
Villarrasa,66.0,11.0,32.0,2176.0,505.514706,1470.588235,0.0
Gibraleón,210.0,71.0,141.0,12607.0,563.179186,1118.426271,5.0
Palma del Condado (La),248.0,62.0,115.0,10761.0,576.154632,1068.673915,5.0
Aracena,155.0,55.0,84.0,8107.0,678.426052,1036.141606,7.0
Calañas,32.0,9.0,27.0,2791.0,322.465066,967.395199,3.0
Valverde del Camino,156.0,48.0,103.0,12820.0,374.414977,803.432137,10.0
Niebla,53.0,17.0,33.0,4117.0,412.922031,801.554530,5.0
Cortegana,121.0,8.0,37.0,4666.0,171.453065,792.970424,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,26.0,1.0,12.0,3073.0,32.541490,390.497885,0.0,0.083333
Cortegana,121.0,8.0,37.0,4666.0,171.453065,792.970424,4.0,0.216216
San Juan del Puerto,96.0,4.0,16.0,9300.0,43.010753,172.043011,0.0,0.250000
Cerro de Andévalo (El),28.0,1.0,4.0,2364.0,42.301184,169.204738,0.0,0.250000
Rosal de la Frontera,15.0,3.0,10.0,1697.0,176.782557,589.275192,0.0,0.300000
Bollullos Par del Condado,165.0,12.0,38.0,14272.0,84.080717,266.255605,5.0,0.315789
Beas,18.0,3.0,9.0,4257.0,70.472163,211.416490,0.0,0.333333
Calañas,32.0,9.0,27.0,2791.0,322.465066,967.395199,3.0,0.333333
San Bartolomé de la Torre,10.0,1.0,3.0,3736.0,26.766595,80.299786,0.0,0.333333
